### MPI Log Parse

This code will take a log file dump from one of our MPI Tests and average the `timeDelta` column for rows with a common `testName`. It will produce a file of the form

```
testName,timeDelta
```

Note: this resulting file may contain rows with duplicate `testName` entries.  In our Baseline test, those duplicates had the exact same value and could be disgarded.

There may be corrupted lines, though their appearance appears to be irregular in our test cases.  The following Regex will help you identify malformed data lines manually in a text editor:

```Regex
^(?!.*,.*,.*,.*,.*,.*).*$
```

In [1]:
import pandas as pd
import numpy as np

In [50]:
testNameColumn = 'testName'
originColumn = 'thisID'
destinationColumn = 'thatID'
timeColumn = 'timeDelta'

inFileName = 'data-source/monteCarloPi/20170501-17-11-mcpi-1000000.csv'
outFileName = 'data-result/mcpi-parsed-1mil-02.csv'

In [51]:
completeData = pd.read_csv(inFileName, comment='#')

# remove lines that aren't data we need
completeData = completeData[completeData[testNameColumn].str.contains('^[0-9]+n*-[0-9]+p*.*$')]
completeData[timeColumn] = completeData[timeColumn].astype(float)

# uncomment to check your input data
# completeData

In [ ]:
testNames = np.unique(completeData[testNameColumn].values)
testNamesOutput = []
averages = []
for testName in testNames:
    data = completeData[completeData[testNameColumn].str.contains(testName)]
    procs = np.unique(data[originColumn].values)
    for proc in procs:
        otherProcs = np.delete(procs, proc)
        for otherProc in otherProcs:
            values = data[(data[originColumn] == proc) & 
                          (data[destinationColumn] == otherProc) | 
                          (data[destinationColumn] == proc) & 
                          (data[originColumn] == otherProc)]
            average = 0.0;
            if values.size > 0:
                average = (float) (values[timeColumn].sum()) / values.size
            testNamesOutput = np.append(testNamesOutput, testName)
            averages = np.append(averages, average)

dfData = {testNameColumn: testNamesOutput, timeColumn: averages}
outputDF = pd.DataFrame(dfData)
outputDF.columns = [testNameColumn, timeColumn]
outputDF.to_csv("" + outFileName)
print(outFileName + " saved")